Welcome to our basic PowerCLI Demo.

Let's start with connecting to our environment.

In [2]:
$vCenterServer = 'flashstack-vcenter.puretec.purestorage.com'
$vCenterUser= 'administrator@vsphere.local'
$vCenterCred = Get-Credential -UserName $vCenterUser
Connect-VIServer -Server $vCenterServer -Credential $vCenterCred -Protocol https -ErrorAction Stop

Now that we're connected, let's get some vSphere objects in PowerShell.

In [3]:
Get-Datacenter


Name
----
DX-Temp
SolutionsLab



In [5]:
Get-Cluster


Name                           HAEnabled  HAFailover DrsEnabled DrsAutomationLevel
                                          Level
----                           ---------  ---------- ---------- ------------------
DX-Cluster                     False      1          False      FullyAutomated
MetroCluster1                  True       1          True       FullyAutomated
Management-Cluster             True       1          True       FullyAutomated



In [6]:
Get-VMHost


Name                 ConnectionState PowerState NumCpu CpuUsageMhz CpuTotalMhz   MemoryUsageGB Memo
                                                                                               ryTo
                                                                                               talG
                                                                                                  B
----                 --------------- ---------- ------ ----------- -----------   ------------- ----
10.21.116.52         Disconnected    Unknown        64           0      127744           0.000 …618
fs-metclus1-esxi01.… Connected       PoweredOn      24         621       57456          31.469 …662
fs-metclus1-esxi03.… Connected       PoweredOn      24          98       57456          20.847 …662
fs-metclus1-esxi02.… Connected       PoweredOn      24        4451       57456          40.836 …662
fs-metclus1-esxi04.… Connected       PoweredOn      24        2060       57456          33.590 …662

What we above is that different information is returned from these objects, obviously from the object types definitions for what is the most useful information.  If we logically know the relationship of these objects, we can determine how to use the pipeline to show information for the specific inventory that we are looking to target.

In [7]:
Get-Cluster -Name 'MetroCluster1' | Get-VMHost


Name                 ConnectionState PowerState NumCpu CpuUsageMhz CpuTotalMhz   MemoryUsageGB Memo
                                                                                               ryTo
                                                                                               talG
                                                                                                  B
----                 --------------- ---------- ------ ----------- -----------   ------------- ----
fs-metclus1-esxi01.… Connected       PoweredOn      24         616       57456          31.470 …662
fs-metclus1-esxi03.… Connected       PoweredOn      24         107       57456          20.847 …662
fs-metclus1-esxi02.… Connected       PoweredOn      24        4332       57456          40.837 …662
fs-metclus1-esxi04.… Connected       PoweredOn      24        2134       57456          33.590 …662



If we do not know how to determine what cmdlets can be piped to each other based on the first object, we can also determine this with some basic cmdlets.

In [9]:
Get-Cluster -Name 'MetroCluster1' | Get-Member


   TypeName: VMware.VimAutomation.ViCore.Impl.V1.Inventory.ClusterImpl

Name                            MemberType Definition
----                            ---------- ----------
ConvertToVersion                Method     T VersionedObjectInterop.ConvertToVersion[T]()
Equals                          Method     bool Equals(System.Object obj)
GetClient                       Method     VMware.VimAutomation.ViCore.Interop.V1.VIAutomation VIO…
GetHashCode                     Method     int GetHashCode()
GetType                         Method     type GetType()
IsConvertableTo                 Method     bool VersionedObjectInterop.IsConvertableTo(type type)
LockUpdates                     Method     void ExtensionData.LockUpdates()
ToString                        Method     string ToString()
UnlockUpdates                   Method     void ExtensionData.UnlockUpdates()
BaseImage                       Property   VMware.VimAutomation.ViCore.Types.V1.Lcm.BaseImage Base…
CollectiveHostManagemen

Now we take the object type returned from our object (a vSphere cluster in this instance), and use this as our `ParameterType` property of `Get-Command`.

In [10]:
Get-Command -ParameterType 'VMware.VimAutomation.ViCore.Impl.V1.Inventory.ClusterImpl'


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Alias           Apply-VMHostProfile                                13.0.0.20… VMware.VimAutomation…
Cmdlet          Add-VMHost                                         13.0.0.20… VMware.VimAutomation…
Cmdlet          Export-LcmClusterDesiredState                      13.0.0.20… VMware.VimAutomation…
Cmdlet          Get-AdvancedSetting                                13.0.0.20… VMware.VimAutomation…
Cmdlet          Get-AlarmDefinition                                13.0.0.20… VMware.VimAutomation…
Cmdlet          Get-Annotation                                     13.0.0.20… VMware.VimAutomation…
Cmdlet          Get-Cluster                                        13.0.0.20… VMware.VimAutomation…
Cmdlet          Get-Datacenter                                     13.0.0.20… VMware.VimAutomation…
Cmdlet          Get-Datastore

Now, as we look to get more performance within our code for woorking with VMware vSphere, we want to use the data that is returned raw from vSphere (similar to a query directly to the API). This which will remove much of the overhead of using the PowerCLI cmdlets which will return a .NET flavored object, plus requires additional formatting to convert to output we would expect from PowerShell.

Before we are ready to rewrite all of our code, we can see this information within the results of the same PowerCLI cmdlets to get our vSphere inventory objects, and this is under the 'ExtensionData' property.

Let's start by getting a vSphere VM and seeing what details are available.

In [11]:
$DemoVM = Get-VM -Name 'record-jump'
$DemoVM.ExtensionData


Capability           : VMware.Vim.VirtualMachineCapability
Config               : VMware.Vim.VirtualMachineConfigInfo
Layout               : VMware.Vim.VirtualMachineFileLayout
LayoutEx             : VMware.Vim.VirtualMachineFileLayoutEx
Storage              : VMware.Vim.VirtualMachineStorageInfo
EnvironmentBrowser   : EnvironmentBrowser-envbrowser-1815
ResourcePool         : ResourcePool-resgroup-804077
ParentVApp           : 
ResourceConfig       : VMware.Vim.ResourceConfigSpec
Runtime              : VMware.Vim.VirtualMachineRuntimeInfo
Guest                : VMware.Vim.GuestInfo
Summary              : VMware.Vim.VirtualMachineSummary
Datastore            : {Datastore-datastore-804098}
Network              : {DistributedVirtualPortgroup-dvportgroup-64}
Snapshot             : 
RootSnapshot         : {}
GuestHeartbeatStatus : green
LinkedView           : 
Parent               : Folder-group-v56
CustomValue          : {}
OverallStatus        : green
ConfigStatus         : green
ConfigI

The results that are returned from the 'ExtensionData' property are very close to the information that is returned from the RestAPI, and these properties are listed in a more categorical view.  Let's look at the information that is returned about the configuration information of the virtual machine.

In [12]:
$DemoVM.ExtensionData.Config


ChangeVersion                : 2023-06-17T01:17:58.472993Z
Modified                     : 1/1/1970 12:00:00 AM
Name                         : record-jump
GuestFullName                : Microsoft Windows 10 (64-bit)
Version                      : vmx-19
Uuid                         : 4223d0fb-f5ff-274a-3113-c812614c4c40
CreateDate                   : 1/13/2023 5:55:33 PM
InstanceUuid                 : 50232c12-483d-ccae-a37f-a81ae3b58d12
NpivNodeWorldWideName        : 
NpivPortWorldWideName        : 
NpivWorldWideNameType        : 
NpivDesiredNodeWwns          : 
NpivDesiredPortWwns          : 
NpivTemporaryDisabled        : True
NpivOnNonRdmDisks            : 
LocationId                   : 564d15c5-6eb3-47c5-cad3-0f93a0d6574d
Template                     : False
GuestId                      : windows9_64Guest
AlternateGuestName           : 
Annotation                   : 
Files                        : VMware.Vim.VirtualMachineFileInfo
Tools                        : VMware.Vim.ToolsC

Now, if we want to return simply the creaton date of the virtual machone, we can obviously return just this property, which will show us how late I was updating my demos this morning...

In [13]:
$DemoVM.ExtensionData.Config.CreateDate


Friday, January 13, 2023 5:55:33 PM



Now, if we want an easier time in seeing what information if available to us about this vSphere inventory object, we can use a fabulous cmdlet from Lee Holmes which is `Show-Object`.  (Found in the 'PowerShellCookbook' module on the PSGallery)

In [ ]:
$DemoVM | Show-Object